# System

* Intel Xeon E5-2630v4
* Asus X99-E WS motherboard
* 2x Nvidia GeForce 1080 Ti Founders

# Ubuntu Setup Notes

I largely followed this [post](https://blog.slavv.com/the-1700-great-deep-learning-box-assembly-setup-and-benchmarks-148c5ebe6415) for the installation and software configuration instructions. 

A few additions are needed, perhaps specifically for the hardware I'm using.

# Partitions

* `/` root drive
* `/home` **NEVER REFORMAT!!**
* `/var` Logs or DB
* `/tmp` 
* `swap space`, 64GB, same as memory

## Update Ubuntu Packages

Also, I needed to **upgrade to the latest distribution linux kernel for the drivers to work**. For this, I followed the instructions [here](https://askubuntu.com/questions/196768/how-to-install-updates-via-command-line) to update the packages for the distrubtions. The commends needed:

    sudo apt-get update        # Fetches the list of available updates
    sudo apt-get upgrade       # Strictly upgrades the current packages
    sudo apt-get dist-upgrade  # Installs updates (new ones)
    

## Mount Apple Time Capsule

[Here](https://askubuntu.com/questions/524328/ubuntu-14-04-how-to-connect-to-apples-time-capsule), and [here](https://ineed.coffee/418/how-to-automatically-mount-and-umount-apple-time-capsule-on-linux/)

Install packages:

    sudo apt-get install cifs-utils
    sudo apt-get install smbclient
    
Update to scripts:

    TIMECAPSULE_IP="192.168.1.64"
    TIMECAPSULE_VOLUME="/Data"
    TIMECAPSULE_PASSWORD=password here

## Ubuntu Package Installation

Refer to [here](https://askubuntu.com/questions/40779/how-do-i-install-a-deb-file-via-the-command-line) for installing packages. 

To install a `.deb` package:

    sudo dpkg -i DEB_PACKAGE
    
To reinstall a package:

    sudo dpkg -i --reinstall DEB_PACKAGE

To resolve package dependency issues:

    sudo apt-get install -f
    
To remove a package:

    sudo dpkg -r PACKAGE_NAME

## Useful Ubuntu Packages

### Disk IO

    sudo apt-get install sysstat
    
Then use `iostat -d 30 /dev/sda5` to monitor `/dev/sda5` with refresh every 30 seconds.

### Etc

    sudo apt-get install curl
    sudo apt-get install git
    
    
## Unity 8

However, it is not supported by Nvidia so not working...

    sudo apt-get update && sudo apt-get dist-upgrade
    sudo apt-get install unity8-desktop-session-mir

# Nvidia Driver / GeForce 1080 Ti Founders

    
## Best Way to Install Cuda + Nvidia Drivers

Follow the instructions [here](http://blog.nelsonliu.me/2017/04/29/installing-and-updating-gtx-1080-ti-cuda-drivers-on-ubuntu/)

### Nvidia Driver

Use [Ubuntu Graphics Drivers PPA](https://launchpad.net/~graphics-drivers/+archive/ubuntu/ppa). Latest v381.22 is available as of 2017-06-15. 

    sudo add-apt-repository ppa:graphics-drivers/ppa  
    sudo apt-get update
    sudo apt-get install nvidia-381
    sudo reboot
    
If this changes the desktop environment, it is because ownership/permission issues in `HOME/.config/`. Runing `chown` and `chgrp -R` would fix it.
    
**Note**: As of Nvidia driver version 384.69 that came out on 2017-08-22, `pytorch` correctly sees cuda with `torch.cuda.is_availalbe()` after installing using Nvidia's scripts. This is done on a fresh Ubuntu install. However, after updating Ubuntu packages and a reboot, system failed to boot into X. This [post](http://tipsonubuntu.com/2017/08/23/nvidia-384-69-released-new-gpu-support-fixes/) is very useful in showing how to roll back and uninstall Nvidia official drivers. 

**Currently the best way is still to use `ppa:graphics-drivers/ppa` to install version 381.**
    
Run `nvidia-smi` to confirm driver version is the latest.

Compile `/usr/local/cuda/samples/1_Utilities/deviceQuery` to test. 

### Cuda 

**Install with runfile, opt out of Nvidia driver installation as it is an older version.**

Choose Y for creating symbolic link for `/usr/local/cuda`.

**Don't forget to set:** `umask 022`

* Add `/usr/local/cuda/bin` to `PATH`
* Add `/usr/local/cuda/lib64` to `LD_LIBRARY_PATH`

## Previous Failed Attempts

A few issues I encountered here:

### Driver Installation Issues, version 381.22

Had problem with this installation, potentially due to the fact that it was a reinstall. In a reinstall, some GL libs are **skipped** and not installed. Solution was to add manual switches to force them to be reinstalled. 

However, I still cannot get the `drm` tools to work. 

The parameters I used:

    sudo ./NVIDIA-Linux-x86_64-381.22.run -install-libglvnd -glvnd-glx-client -glvnd-egl-client -no-drm


### PCIe Bus ERROR

This was written for Nvidia drvier version 381.22.

This seems to be a problem for Asus, X99 chipset & PLX switches issue, according to this [post](https://forums.geforce.com/default/topic/957456/geforce-drivers/gtx-1080-throwing-bad-tlp-pcie-bus-errors/2/). 

I used one of the suggestions by adding this kernel parameters: `pcie_aspm=off`. This seems to have worked for me so far, no more error messages. **Note: this is causing problem at start up for Nvidia driver v 384.69.**

Follow this [page](https://askubuntu.com/questions/19486/how-do-i-add-a-kernel-boot-parameter) on how to add kernel boot options.

## Others    

### Disable Secure Boot

Some posts suggested disabling secure book in BIOS helped with the `drm` issue, but this did not work for me.

## Software Updates

### Pytorch, HDBSCAN, Joblib, xarray, etc

    conda install pytorch torchvision cuda80 -c soumith
    conda install -c conda-forge hdbscan tabulate
    conda install -c defaults joblib xarray dask netCDF4 bottleneck holoviews colorcet tqdm
    conda install -c conda-forge xarray cartopy pynio # for difficult to build cartopy and pynio.
    conda install -c districtdatalabs yellowbrick
    conda install -c bokeh datashader
    
    pip install hypertools
    pip install git+https://github.com/bashtage/arch.git
    pip install --upgrade google-api-python-client
    
    # filterpy
    git clone https://github.com/rlabbe/filterpy.git
    cd filterpy
    python setup.py install

### Jupyter Notebook 

#### Password

To setup a password for jupyter server, see [here](https://jupyter-notebook.readthedocs.io/en/stable/public_server.html)

#### Crontab

Useful tip from this [link](https://stackoverflow.com/questions/2229825/where-can-i-set-environment-variables-that-crontab-will-use)

Instead of calling `jupyter notebook` directly in the crontab scripts, call a script that sets up the needed environment variables, such as `PYTHONPATH`, to start `jupyter notebook` server. See [this script](./cron_jupyter.sh).

`Crontab` startup logs can be found in `/var/log/syslog`

Step:
    1. sudo crontab -e
    2. insert line: `@reboot \home\zwl\.cron_jupyter.sh`
    3. Save and exit

### Rethinking / R

Create `.Rprofile` file and write the line below for MRO.

    options(repos = c(CRAN = "https://cran.revolutionanalytics.com"))
    
`rstan` Installation [here](https://github.com/stan-dev/rstan/wiki/Installing-RStan-on-Mac-or-Linux)

First run this, some packages needs openssl headers.

    sudo apt-get install libssl-dev
    
Then run in `R`:

    install.packages("rstan", repos = "https://cloud.r-project.org/", dependencies=TRUE)
    
Installing `Rethinking`:

    install.packages(c("coda","mvtnorm","devtools","loo"))
    library(devtools)
    devtools::install_github("rmcelreath/rethinking")
    
Might need to do a local install:

    git clone https://github.com/rmcelreath/rethinking.git
    library(devtools)
    devtools::install('/home/zwl/git/rethinking/')

### TA-Lib

Need to install `TA-Lib` c libraries first. Follow instructions on [github](https://github.com/mrjbq7/ta-lib).

Download library [here](http://www.ta-lib.org/hdr_dw.html)

    untar and cd
    ./configure --prefix=/usr
    make
    umask 022
    sudo make install
    
    pip install TA-Lib

## Sublime Text 3

User settings / key bindings can be found in Gmail. Labelled as `Code`.

Instructions [here](https://realpython.com/blog/python/setting-up-sublime-text-3-for-full-stack-python-development/)

Packages installed:

    Anaconda
    AutoDocstring
    GitGutter
    Side bar
    Markdown Preview
    Package Control
    Python PEP8 Autoformat
    SublimeLinter

# Chrome Menu Bar Size

Follow instructions on this [page](https://superuser.com/questions/1116767/chrome-ui-size-zoom-levels-in-ubuntu-16-04)

    sudo gedit /usr/share/applications/google-chrome.desktop
    
Change the following line:

    Exec=/usr/bin/google-chrome-stable --force-device-scale-factor=1.5 %U